In [12]:
import pandas as pd
%config IPCompleter.greedy = True

In [13]:
datapath="purchase_data.csv"
data_file=pd.read_csv(datapath)
data_file.head()
data_file = data_file.dropna(how="any")
data_file.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",$3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,$1.56
2,2,Ithergue48,24,Male,92,Final Critic,$4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,$3.27
4,4,Iskosia90,23,Male,131,Fury,$1.44


In [ ]:
uniqueItem = len(data_file["SN"].unique())
totalUniqueItem=len(data_file["Item ID"].unique())
avgPrice=data_file["Price"].mean()
totalPurchaseID=len(data_file["Purchase ID"].unique())
totalRevenue=data_file["Price"].sum()
# create the data frane

raw_data = {'Number of Unique Items': totalUniqueItem,
'Average Price': f'${round(avgPrice,2)}',
'Number of Purchases':totalPurchaseID,
'Total Revenue':f'${round(totalRevenue,2)}'}

df = pd.DataFrame(raw_data,index = [0])
df.head()   

In [ ]:
#Create a new dataframe and clean the duplicate

New_index = data_file[["SN","Gender"]]

New_index=New_index.drop_duplicates()

#Group by gender
grouped_Gender= New_index.groupby(['Gender'])
#Calculate the Gender total percentage
total_percentage=round(grouped_Gender.count() / New_index.count() *100,2)
total_percentage=total_percentage.drop(['Gender'], axis=1)
#Calculate the total count s
total_count=grouped_Gender.count()
#Merge percentage and total counts into new columns 
Gender_analysis= pd.merge(total_count, total_percentage, on="Gender")

Gender_analysis=Gender_analysis.rename(columns={"SN_x":"Total Count",
                                                "SN_y":"Percentage of Players","Gender": " "})

Gender_analysis


In [ ]:
#Purchasing Analysis (Gender)
#Create first group by gender
groupByGender= data_file.groupby(['Gender'])
#Now group by Gender and SN so that we can calulated one player multiple purchase
groupByGender_Per= data_file.groupby(['Gender','SN'])
#Group by gender for unique player
Total_Avg_Purchase_Person=(pd.DataFrame(groupByGender["Gender"].count())).rename({'Gender':'Purchase Count'}, axis='columns')
Total_Avg_Purchase_Person['Total Purchase Value']=groupByGender["Price"].sum()
Total_Avg_Purchase_Person['Average Purchase Price']=round(groupByGender["Price"].sum()/groupByGender["Price"].count(),2)
Total_Avg_Purchase_Person['Avg Total Purchase per Person']=round(Total_Avg_Purchase_Person['Total Purchase Value']/\
                                                                 data_file.groupby(['Gender'])['SN'].nunique(),2)
Total_Avg_Purchase_Person['Avg Total Purchase per Person']=Total_Avg_Purchase_Person['Avg Total Purchase per Person'].map("${:,.2f}".format)
Total_Avg_Purchase_Person['Average Purchase Price']=Total_Avg_Purchase_Person['Average Purchase Price'].map("${:,.2f}".format)  
Total_Avg_Purchase_Person.head()                                                        



In [ ]:
#Create a new dataframe and clean the duplicate

New_index = data_file[["SN","Gender"]]

New_index=New_index.drop_duplicates()

#Group by gender
grouped_Gender= New_index.groupby(['Gender'])
#Calculate the Gender total percentage
total_percentage=round(grouped_Gender.count() / New_index.count() *100,2)
total_percentage=total_percentage.drop(['Gender'], axis=1)
#Calculate the total count s
total_count=grouped_Gender.count()
#Merge percentage and total counts into new columns 
Gender_analysis= pd.merge(total_count, total_percentage, on="Gender")

Gender_analysis=Gender_analysis.rename(columns={"SN_x":"Total Count",
                                                "SN_y":"Percentage of Players","Gender": " "})

Gender_analysis

In [ ]:
#Age Demographics Establish bins for ages
bins=[0,9,14,19,24,29,34,39,100]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29","30-34","35-39","+40"]
New_df_Age = data_file[["Age","SN"]]
New_df_Age=New_df_Age.drop_duplicates()
New_df_Age["Counts"]=pd.cut(New_df_Age["Age"], bins, labels=group_names)
New_age=New_df_Age.groupby(['Counts'])
New_age_counts=New_age.count()
New_age_percent=pd.DataFrame(New_age_counts["Age"])
New_age_percent['Percentage of Players']=round((New_age_counts["Age"]/New_age_counts["Age"].sum() * 100),2)
New_age_percent=New_age_percent.rename(columns={"Age":"Total Count"})
New_age_percent




In [ ]:
 #Purchasing Analysis (Age)
bins=[0,9,14,19,24,29,34,39,100]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29","30-34","35-39","40+"]
#Create new source frame from main frame 
df_Age = data_file[["Age","SN","Purchase ID","Price"]]
df_Age=df_Age.drop_duplicates()
df_Age["Total Count"]=pd.cut(df_Age["Age"], bins, labels=group_names)
df_age=df_Age.groupby(['Total Count'])    
df_final_age=pd.DataFrame(df_age['Purchase ID'].count())
df_final_age['Average Purchase Price']=round(df_age['Price'].sum()/df_age['Purchase ID'].count(),2)
df_final_age['Total Purchase Value']=df_age['Price'].sum()
df_final_age['Avg Total Purchase per Person']=round(df_age['Price'].sum()/df_age['SN'].nunique(),2)
df_final_age=df_final_age.reset_index()

sort_order = {"10-14":0,"15-19":1,"20-24":2,"25-29":3,"30-34":4,"35-39":5,"40+":6, "<10":7}

df_final_age['sortOrder']=df_final_age['Total Count'].map(sort_order)

df_final_age.sort_values("sortOrder",inplace=True)

df_final_age.set_index("Total Count",inplace=True)

df_final_age=df_final_age.rename_axis(None)

df_final_age=df_final_age[['Purchase ID', 'Average Purchase Price', 'Total Purchase Value',
       'Avg Total Purchase per Person']].rename(columns={"Purchase ID":"Purchase Count"})
df_final_age['Average Purchase Price']=df_final_age['Average Purchase Price'].map("${:,.2f}".format)

df_final_age['Total Purchase Value']=df_final_age['Total Purchase Value'].map("${:,.2f}".format)

df_final_age['Avg Total Purchase per Person']=df_final_age['Avg Total Purchase per Person'].map("${:,.2f}".format)

df_final_age

In [ ]:
# Top Spenders

groupby_SN=data_file.groupby(['SN'])

final_results=(pd.DataFrame(groupby_SN['Purchase ID'].count())).rename({'Purchase ID':'Purchase Count'},axis='columns')

final_results['Average Purchase Price']=round(groupby_SN['Price'].sum()/groupby_SN['Price'].count(),2)

final_results['Total Purchase Value']=round(groupby_SN['Price'].sum(),2)

final_results= final_results.sort_values(by=['Total Purchase Value'],ascending=False)

final_results['Average Purchase Price']=final_results['Average Purchase Price'].map("${:,.2f}".format)
final_results['Total Purchase Value']=final_results['Total Purchase Value'].map("${:,.2f}".format)

final_results.head()

In [14]:
# Most Popular Items
groupby_Item=data_file.groupby(['Item ID','Item Name'])

pop_item=pd.DataFrame(groupby_Item['Purchase ID'].count()).rename({'Purchase ID':'Purchase Count'},axis='columns')
pop_item['Item Price']=groupby_Item['Price'].min()
pop_item['Total Purchase Value']=pop_item['Purchase Count'] * pop_item['Item Price']
pop_item= pop_item.sort_values(by=['Purchase Count'],ascending=False)
pd.options.display.float_format = '${:,.2f}'.format
pop_item.head()


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [15]:
#Most Profitable Items use the prvious dataframe 
pop_item= pop_item.sort_values(by=['Total Purchase Value'],ascending=False)
pd.options.display.float_format = '${:,.2f}'.format
pop_item.head()


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
